In [31]:
import pandas as pd
import os
import pyexcel as pe

In [27]:
ALLOWED_FILE_TYPES = ("xlsx", "xls")
CHUNK_SIZE = 50

# creating required paths and dir
CWD = os.getcwd()
INPUT_DIR = os.path.join(CWD, "input files")
OUTPUT_DIR = os.path.join(CWD, "output files")

if not os.path.exists(INPUT_DIR):
    logger.info("input files dir does not exists, Creating..")
    os.mkdir(INPUT_DIR)
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [32]:
def read_excel_chunks(input_file, chunksize):
    print("Loading excel file..")
    records1 = pe.iget_array(file_name=input_file)
    print("Loaded excel file")
    temp = []
    col_names = []
    for i in records1:
        if not col_names:
            col_names = i
            continue
        temp.append(i)
        if len(temp) >= chunksize:
            yield pd.DataFrame(data=temp, columns=col_names)
            temp = []
    pe.free_resources()

In [ ]:
all_excel_files = [i for i in os.listdir(INPUT_DIR) if i.rsplit(os.extsep, 1)[1] in ALLOWED_FILE_TYPES]
print(f"Found {len(all_excel_files)} excel files..")
for file in all_excel_files:
    print(f"Processing {file} file")
    filename, ext = file.rsplit(os.extsep, 1)
    file_path = os.path.join(INPUT_DIR, file)
    file_output_dir = os.path.join(OUTPUT_DIR, f"{filename}_{ext}")
    if not os.path.exists(file_output_dir):
        os.mkdir(file_output_dir)
    for i, df in enumerate(read_excel_chunks(file_path, CHUNK_SIZE)):
        chunk_name = f"{filename}_{i}.xlsx"
        df.to_excel(os.path.join(file_output_dir, chunk_name), index=False)
        print(f"Saved {chunk_name}")